This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X?api_key='+API_KEY

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
import json

r = requests.get(url)
json_data = r.json()
prettyJson = json.dumps(json_data, indent = 4)
#print(prettyJson)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY
r = requests.get(url)
#print(type(r))

In [6]:
# 2. Convert the returned JSON object into a Python dictionary.

json_data_2017 = r.json()
print(type(json_data_2017))

<class 'dict'>


In [7]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.

data = json_data_2017['dataset']['data']
open_col_idx = json_data_2017['dataset']['column_names'].index('Open')
opening_price = [row[open_col_idx] for row in data]

highest = max(i for i in opening_price if i is not None)
print(highest)
lowest = min(i for i in opening_price if i is not None)
print(lowest)

53.11
34.0


In [8]:
# 4. What was the largest change in any one day (based on High and Low price)?

high_col_idx = json_data_2017['dataset']['column_names'].index('High')
low_col_idx  = json_data_2017['dataset']['column_names'].index('Low')

change_daily = [row[high_col_idx]-row[low_col_idx] for row in data]

largest_change = max(i for i in change_daily if i is not None)

largest_change

2.8100000000000023

In [9]:
# 5. What was the largest change between any two days (based on Closing Price)?

close_col_idx = json_data_2017['dataset']['column_names'].index('Close')

closing_price = [row[close_col_idx] if row[close_col_idx] is not None else float('NAN') for row in data]

change_2days  = [j - i for i, j  in zip(closing_price[:-1], closing_price[1:])]

largest_2days = max(change_2days)
                 
largest_2days                 

2.559999999999995

In [10]:
# 6. What was the average daily trading volume during this year?

trd_vol_idx = json_data_2017['dataset']['column_names'].index('Traded Volume')

trading_volume = [row[trd_vol_idx] for row in data if row[trd_vol_idx] is not None]

average_trd_vol = sum(trading_volume)/len(trading_volume)

average_trd_vol

89124.33725490196

In [11]:
# 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

trading_volume.sort()

L = len(trading_volume)

if (L % 2) != 0:
    median_trd_vol = trading_volume[L//2]
else:
    median_trd_vol = (trading_volume[L//2] + trading_volume[L//2 - 1])/2

median_trd_vol    

76286.0